In [1]:
import tabula
import textract
import re
import pymysql
from os import listdir
from os.path import isfile, join
import pandas as pd
from IPython.display import display
import db_conn

In [2]:
CONN = db_conn.get_connection()

In [7]:
def get_player_info(player_info):
    print(player_info)
    team_id = player_info['team_id']
    name = player_info['name']
    
    curs = CONN.cursor()
    find_sql = """select id from player where lower(name) like %s and team_id=%s"""
    curs.execute(find_sql, ('%'+name+'%', team_id))
    player_id = curs.fetchone()
    return player_id

def get_player_id(player_info):
#     print(player_info)
    team_id = player_info['team_id']
    num = player_info['number']
    
    curs = CONN.cursor()
    find_sql = """select id from player where num=%s and team_id=%s"""
    curs.execute(find_sql, (num, team_id))
    player_id = curs.fetchone()
    return player_id

def insert_player_stat(player_stat):
    print(player_stat)
    curs = CONN.cursor()
    insert_sql = """insert into player_stat (player_id, match_id, stat_type, stat_value, half) values (%s, %s, %s, %s, %s)"""
    curs.execute(insert_sql, (player_stat['player_id'], player_stat['match_id'], player_stat['stat_type'], player_stat['stat_value'], player_stat['half'],))
    CONN.commit()
    
def save_passing_stat(player_passing):
    player_id = get_player_id({'team_id':player_passing['team_id'], 'number':player_passing['number']})
    
    stat_types = ['Long_PC', 'Long_PA', 'Medium_PC', 'Medium_PA', 'Short_PC', 'Short_PA', 'Total_PC', 'Total_PA', 'Total_PP']
    for s_type in stat_types:
        insert_player_stat({'player_id':player_id, 'match_id':player_passing['match_id'], 'stat_type':s_type, 'stat_value':player_passing[s_type], 'half':'total'})

In [4]:
root_dirs =['/Users/gracelee/football_worldcup', #group phase
            '/Users/gracelee/football_worldcup_knock']#knockout phase

In [5]:
countries = {'russia':'1',
            'saudi arabia':'2',
            'egypt':'3',
            'uruguay':'4',
            'portugal':'5',
            'spain':'6',
            'morocco':'7',
            'ir iran':'8',
            'france':'9',
            'australia':'10',
            'peru':'11',
            'denmark':'12',
            'argentina':'13',
            'iceland':'14',
            'croatia':'15',
            'nigeria':'16',
            'brazil':'17',
            'switzerland':'18',
            'costa rica':'19',
            'serbia':'20',
            'germany':'21',
            'mexico':'22',
            'sweden':'23',
            'korea republic':'24',
            'belgium':'25',
            'panama':'26',
            'tunisia':'27',
            'england':'28',
            'poland':'29',
            'senegal':'30',
            'colombia':'31',
            'japan':'32'}

In [ ]:
regex = re.search('([0-9]+)', 'ABDULLAH ALMUAIOUF 1')
regex.group(0)

In [ ]:
for _dir in root_dirs:
    onlyfiles = [f for f in listdir(_dir) if isfile(join(_dir, f))]
    onlyfiles = [_dir+'/'+x for x in onlyfiles if ('passingdistribution' in x)]
    
    for z in range(len(onlyfiles)):
        print('file z::: %s' % str(z))
        print(onlyfiles[z])
        player_statistic = textract.process(onlyfiles[z])
        lines = str(player_statistic).split('\\n')
#         print(lines[6])
        country_one = lines[6].split('-')[0].strip().lower()
        country_two = re.search('([A-z ]+)', lines[6].split('-')[1]).group(0).strip().lower()
#         print(country_one)
#         print(country_two)
#         print(lines[9])
        country_ids = [countries[country_one], countries[country_two]]
        countries_in = []
        if lines[7].find('#')>=0:
            match_id = lines[7].split(' ')[1]
        else:
            match_id = lines[11].split(' ')[1]
#         print('match id %s' % (str(match_id)))

        dfs = tabula.read_pdf(onlyfiles[z], multiple_tables=True)
        for i in range(2):
            row_len = 14
            if dfs[i].shape[0]<14:
                row_len = dfs[i].shape[0]
            sample = dfs[i].iloc[[i for i in range(0, row_len)]]

            if len(dfs[i].columns)==25:
                sample =sample[[0,1,16,17,18,19,20,21,22,23,24]]
            elif len(dfs[i].columns)==26:
                sample=sample[[0,1,17,18,19,20,21,22,23,24,25]]
            elif len(dfs[i].columns)==27:
                sample=sample[[0,1,18,19,20,21,22,23,24,25,26]]
            else:
                print('UNEXPECTED!!!!!')
                display(sample)

            sample.columns = ['name', 'number', 'Long_PC', 'Long_PA', 'Medium_PC', 'Medium_PA', 'Short_PC', 'Short_PA', 'Total_PC', 'Total_PA', 'Total_PP']

#             check country id
#             player_name  = sample.iloc[3]['name'].lower().strip()
#             player_name = player_name[:len(player_name)-2].strip()
#             player = get_player_info({'name':player_name, 'team_id':country_ids[0]})
#             if player is None:
#                 player = get_player_info({'name':player_name, 'team_id':country_ids[1]})
#                 if player is None:
#                     raise Exception('none of team is found for player %s' % player_name)
#                 else:
#                     country_id = country_ids[1]
#             else:
#                 country_id = country_ids[0]
            
#             countries_in.append(country_id)
#         if countries_in[0]==countries_in[1]:
#             print('EXCEPTION=========================== same country')
#         print(countries_in)
            country_id = country_ids[i]

            for ind, row in sample.iterrows():
                if row['name'].find('Total') >=0:
                    continue
                if row['number'].isdigit():
                    number = row['number']
                else:
                    number = re.search('([0-9]+)', row['name']).group(0)

                name = row['name'].replace(number,'').strip()
                player_stat = {
                    'team_id':country_id,
                    'match_id':match_id,
                    'name':name,
                    'number':number,
                    'Long_PC':row['Long_PC'],
                    'Long_PA':row['Long_PA'],
                    'Medium_PC':row['Medium_PC'],
                    'Medium_PA':row['Medium_PA'],
                    'Short_PC':row['Short_PC'],
                    'Short_PA':row['Short_PA'],
                    'Total_PC':row['Total_PC'],
                    'Total_PA':row['Total_PA'],
                    'Total_PP':row['Total_PP']
                }
#                 print(player_stat)
                save_passing_stat(player_stat)

In [ ]:
should_num = 28
is_done = False
for i in range(len(lines)):

#     if lines[i].find('From')>=0:
#         is_start = True
    
    if !is_done and re.search('[a-zA-Z ] - ', lines[i]) is not None:

#         print(lines[i])